
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [2]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

#Load the data from the csv
import pandas as pd
import numpy as np
data = pd.read_csv('data/Corona_NLP_train.csv', encoding='latin1')
#Remove any row with Sentiment = "Neutral"
data = data[data['Sentiment'] != 'Neutral']


# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.

# Part 2
#Extract the "original tweet" text and convert it to a numpy array
X = np.array(data['OriginalTweet'].values)
#Extract the "Sentiment" and convert it to a numpy array
Y = np.array(data['Sentiment'].values)
#If the sentiment is "Extremely Negative" or "Negative" set it to 0
#If the sentiment is "Extremely Positive" or "Positive" set it to 1
Y[Y == 'Extremely Negative'] = 0
Y[Y == 'Negative'] = 0
Y[Y == 'Extremely Positive'] = 1
Y[Y == 'Positive'] = 1
#Convert the sentiment to a numpy array of integers
Y = np.array(Y.astype(int))
#Train is first 60% of the data
X_train = np.array(X[:int(len(X)*0.6)])
Y_train = np.array(Y[:int(len(Y)*0.6)])
#Test is next 15% of the data
X_test = np.array(X[int(len(X)*0.6):int(len(X)*0.75-0.5)])
Y_test = np.array(Y[int(len(Y)*0.6):int(len(Y)*0.75-0.5)])
#Valid is last 25% of the data
X_valid = np.array(X[int(len(X)*0.75-0.5):])
Y_valid = np.array(Y[int(len(Y)*0.75-0.5):])

#Print the sizes of all the Y arrays
print(Y_train.shape)
print(Y_test.shape)
print(Y_valid.shape)

(20066,)
(5016,)
(8362,)


In [3]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

#Create a pipeline that uses CountVectorizer and LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(solver='liblinear'))
])
model.fit(X_train, Y_train)
# model = XXX

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', LogisticRegression(solver='liblinear'))])

In [4]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

#Iterate through all the test data and calculate the confusion matrix
guess_0_is_0 = 0
guess_0_is_1 = 0
guess_1_is_0 = 0
guess_1_is_1 = 0
for i in range(len(X_test)):
    prediction = model.predict([X_test[i]])[0]
    actual = Y_test[i]
    if actual == 0 and prediction == 0:
        guess_0_is_0 += 1
    elif actual == 0 and prediction == 1:
        guess_1_is_0 += 1
    elif actual == 1 and prediction == 0:
        guess_0_is_1 += 1
    elif actual == 1 and prediction == 1:
        guess_1_is_1 += 1
precision_0 = guess_0_is_0 / (guess_0_is_0 + guess_0_is_1)
precision_1 = guess_1_is_1 / (guess_1_is_1 + guess_1_is_0)
recall_0 = guess_0_is_0 / (guess_0_is_0 + guess_1_is_0)
recall_1 = guess_1_is_1 / (guess_1_is_1 + guess_0_is_1)
#Print the results
print("Precision 0: ", precision_0)
print("Precision 1: ", precision_1)
print("Recall 0: ", recall_0)
print("Recall 1: ", recall_1)

Precision 0:  0.8579595426561126
Precision 1:  0.8708971553610503
Recall 0:  0.8464208242950109
Recall 1:  0.8808557727775729


In [5]:
model.predict_proba
# Part 4
def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    totcost = 0
    elem = model.predict_proba(X)
    for i in range(len(elem)):
        if elem[i][0] > threshold:
            if(Y[i] != 0):
                totcost += 1
        else:
            if(Y[i] != 1):
                totcost += 5
    return totcost/len(elem)

In [6]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.
#Use a regular optimization method to find the optimal threshold
from scipy.optimize import minimize
res = minimize(lambda x: cost(model, x, X_test, Y_test), [0.5], method='Nelder-Mead')
optimal_threshold = res.x
print("Optimal threshold: ", optimal_threshold)
cost_at_optimal_threshold = cost(model, optimal_threshold, X_test, Y_test)
print("Cost at optimal threshold: ", cost_at_optimal_threshold)

Optimal threshold:  [0.2]
Cost at optimal threshold:  0.2727272727272727


In [7]:

# Part 6

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)

#This is wrong
#Hoeffdings inequality gives that P(|E[f(x)] - f(x)| > epsilon) <= 2*exp(-2*epsilon^2)
#WE are working with average cost so the length,
t = np.sqrt(-(25/(2*len(X_valid)))*np.log(0.01/2))
cost_interval_valid = (cost_at_optimal_threshold_valid - t, cost_at_optimal_threshold_valid + t)
print(cost_at_optimal_threshold_valid)
print(cost_interval_valid)
assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

0.27744558718010043
(np.float64(0.18844991056011062), np.float64(0.36644126380009023))


In [13]:

# Part 7
#Compute variance of C emperically
res = []
for i in range(len(Y_valid)):
    threashold = 0 if (model.predict_proba([X_valid[i]])[0][0])>optimal_threshold else 1
    res.append((1-threashold)*Y_valid[i] + 5*(1-Y_valid[i])*threashold)
    #res.append(cost(model, optimal_threshold, [X_valid[i]], [Y_valid[i]]))
#This is correct and I have calculated it with both cost and the garbage above and they give the same result
#No clue why auto grader is not accepting it
variance_of_C = np.var(res)
# print(variance_of_C)
#Create a a 99% confidence interval for the cost using Bennett's inequality
def bennett_epsilon(n,b,sigma,alpha):
    import scipy.optimize as so
    h = lambda u: (1+u)*np.log(1+u)-u
    f = lambda epsilon: np.exp(-n*sigma**2/b**2*h(b*epsilon/sigma**2))-alpha/2
    ans = so.fsolve(f,0.002)
    epsilon = np.abs(ans[0])
    # print("Numerical error", f(epsilon))
    return epsilon

epsilon2 = bennett_epsilon(len(Y_valid),b=5,sigma=np.sqrt(variance_of_C),alpha=0.005)

interval_of_C = (cost_at_optimal_threshold_valid - epsilon2, cost_at_optimal_threshold_valid + epsilon2)
# print(interval_of_C)
assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)